In [2]:
# normalized VGG16 feature data fed into simple LSTM

# First, import necessary libraries
import tensorflow as tf
import numpy as np
import keras.backend as K

/Users/chloeloughridge/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
Using TensorFlow backend.


In [3]:
# setting up the keras stuff
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, BatchNormalization
from keras.layers import LSTM
# my custom data_utils file
from data_utils_local08 import *

In [4]:
# uploading the X values
X_input = np.load('../Desktop/VGG16_feature_data.npy') #VGG16 data
#X_input = np.load('VGG16.npy')
# X_input = np.load('../Desktop/InceptionResNetV2_feature_data.npy') #resnet data!

#X_input = np.load('../Desktop/audio_feature_data.npy') #audio feature data!

print(X_input.shape)

(7, 212, 4096)


In [5]:
# uploading the Y values
y_data_input = np.load('../Desktop/labels.npy')
#y_data_input = np.load('labels.npy')
print(y_data_input.shape)

(7, 219)


In [6]:
timesteps = X_input.shape[1]
data_dim = X_input.shape[2]

X_train = X_input[:4,:timesteps, :]
Y_train = y_data_input[:4, :timesteps]

X_test = X_input[4:, :timesteps, :]
Y_test = y_data_input[4:, :timesteps]

print(X_test.shape, X_train.shape, Y_train.shape, Y_test.shape)

(3, 212, 4096) (4, 212, 4096) (4, 212) (3, 212)


In [7]:
# constructing a many-to-one LSTM model in keras 
model = Sequential()

# normalization
model.add(BatchNormalization(input_shape=(timesteps, data_dim)))

model.add(LSTM(timesteps, return_sequences=True))
#input_shape=(timesteps, data_dim)

model.add(Flatten()) 
# add the final dense layer and then softmax
model.add(Dense(timesteps, activation='sigmoid'))
# going to add a softmax activation to this
#model.add(Activation('softmax'))

In [8]:
# a new metric of evaluation! the F-score!
def FScore2(y_true, y_pred):
    '''
    The F score, beta=2
    '''
    B2 = K.variable(4)
    OnePlusB2 = K.variable(5)
    pred = K.round(y_pred)
    tp = K.sum(K.cast(K.less(K.abs(pred - K.clip(y_true, .5, 1.)), 0.01), 'float32'), -1)
    fp = K.sum(K.cast(K.greater(pred - y_true, 0.1), 'float32'), -1)
    fn = K.sum(K.cast(K.less(pred - y_true, -0.1), 'float32'), -1)

    f2 = OnePlusB2 * tp / (OnePlusB2 * tp + B2 * fn + fp)

    return K.mean(f2)

In [9]:
# compiling LSTM model
# note that Ng used an Adam optimizer and categorical cross-entropy loss
# but this is a binary classification problem so I think the parameters below should suffice
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['binary_accuracy', FScore2])

In [10]:
# running the LSTM model
model.fit(X_train, Y_train, epochs = 20, batch_size = 128, validation_data=(X_test, Y_test))
print("finished training!")

Train on 4 samples, validate on 3 samples
Epoch 1/20
4/4 [==============================] - 6s 1s/step - loss: 0.7222 - binary_accuracy: 0.5071 - FScore2: 0.2710 - val_loss: 0.9030 - val_binary_accuracy: 0.6604 - val_FScore2: 0.0684
Epoch 2/20
4/4 [==============================] - 3s 848ms/step - loss: 0.0107 - binary_accuracy: 1.0000 - FScore2: 1.0000 - val_loss: 1.9315 - val_binary_accuracy: 0.3318 - val_FScore2: 0.5187
Epoch 3/20
4/4 [==============================] - 4s 892ms/step - loss: 0.0943 - binary_accuracy: 0.9575 - FScore2: 0.8973 - val_loss: 2.6987 - val_binary_accuracy: 0.6148 - val_FScore2: 0.2516
Epoch 4/20
4/4 [==============================] - 3s 860ms/step - loss: 0.0020 - binary_accuracy: 0.9988 - FScore2: 0.9773 - val_loss: 2.4815 - val_binary_accuracy: 0.6242 - val_FScore2: 0.2516
Epoch 5/20
4/4 [==============================] - 4s 898ms/step - loss: 2.1068e-05 - binary_accuracy: 1.0000 - FScore2: 1.0000 - val_loss: 2.4731 - val_binary_accuracy: 0.6274 - val_FSc

In [11]:
model.save('./VGG16_norm_lstm')

In [29]:
# trying to view the model output
out = model.predict(X_train)

movie_index = 1

print("model prediction:")
print(out[movie_index])
print("target:")
print(Y_train[movie_index])

print("before 64:")
print(out[movie_index][63])
print("64:")
print(out[movie_index][65])

print("rounded")
print(np.round(out)[movie_index])

model prediction:
[1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
 1.9221041e-28 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 2.1595651e-19
 1.0000000e+00 1.0000000e+00 2.3602658e-26 1.0000000e+00 1.0000000e+00
 4.4973351e-18 3.1209957e-32 1.9935814e-32 3.3743915e-34 1.0000000e+00
 1.0000000e+00 2.1284409e-12 1.6664713e-14 1.0000000e+00 1.0000000e+00
 1.0000000e+00 1.0000000e+00 1.7663178e-08 1.0000000e+00 1.0000000e+00
 1.0000000e+00 1.0000000e+00 2.7702462e-30 2.5049629e-30 1.0000000e+00
 2.6652263e-31 4.6479136e-31 2.5350656e-34 4.2515438e-35 4.0209436e-32
 1.0000000e+00 9.4842254e-15 1.0000000e+00 1.4375714e-11 1.0000000e+00
 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.